In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/home/davcic/CP_Testing')
sys.path.append('/home/davcic/CP_Testing/HDPHMM')

import hdphmm
from hdphmm import generate_timeseries as gent
import numpy as np
import matplotlib.pyplot as plt
import generate_timeseries as gent2

Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.

****** PyMBAR will use 64-bit JAX! *******
* JAX is currently set to 32-bit bitsize *
* which is its default.                  *
*                                        *
* PyMBAR requires 64-bit mode and WILL   *
* enable JAX's 64-bit mode when called.  *
*                                        *
* This MAY cause problems with other     *
* Uses of JAX in the same code.          *
******************************************



In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''
print(f'Selected CPU')

import jax
import jax.random as jrnd
import jax.numpy as jnp
import distrax as dx
import jaxkern as jk

# from jax.config import config
# config.update("jax_enable_x64", True)  # crucial for Gaussian processes

from tensorflow_probability.substrates import jax as tfp
tfd = tfp.distributions
tfb = tfp.bijectors


sys.path.append('/home/davcic/CP_Testing')

from New_kernel_1 import Discontinuous_multiple
from New_kernel_1 import Discontinuous_multiple_params
from New_kernel_1 import Discontinuous_multiple_params_hyper
from Poisson_Process_added import Poisson_Process_hyper
from Uniform_modified import Uniform_mod
from Normal_modified import LogNormal_mod
import gc

sys.path.append('/home/davcic/CP_Testing/Classes')

from GP_CP import GP_CP_Marginal, GP_CP_Latent

name = 'Toy_dataset_RBF_3'

Selected CPU


2024-06-20 23:13:10.688508: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


## Loading GPs

In [16]:
num_runs = 3

GP_marginal = []
GP_latent = []

for i in range(num_runs):
    GP_marginal.append(jnp.load(name + f'/GP_Marginal/GP_marginal_trained_{i}.npy', allow_pickle = True)[()])
    GP_latent.append(jnp.load(name + f'/GP_Latent/GP_latent_trained_{i}.npy', allow_pickle = True)[()])



In [17]:
ground_truth = jnp.load(name + f'/ground_truth.npy', allow_pickle = True)[()]

In [19]:
GP_marginal[0].jaccard_metric(ground_truth)

Array(0.9524, dtype=float32)

In [13]:
key = jrnd.PRNGKey(1234)
number_metric = jnp.zeros((2, 3))
location_metric = jnp.zeros((2, 3))
likelihood_metric = jnp.zeros((2, 3))
jaccard_metric = jnp.zeros((2, 3))

for i in range(num_runs):
    jaccard_metric = jaccard_metric.at[0, i].set(GP_marginal[i].jaccard_metric(ground_truth))
    number_metric = number_metric.at[0, i].set(GP_marginal[i].number_metric(ground_truth))
    location_metric = location_metric.at[0, i].set(GP_marginal[i].location_metric(ground_truth))
    likelihood_metric = likelihood_metric.at[0, i].set(GP_marginal[i].likelihood_metric(key))
    number_metric = number_metric.at[1, i].set(GP_latent[i].number_metric(ground_truth))
    location_metric = location_metric.at[1, i].set(GP_latent[i].location_metric(ground_truth))
    likelihood_metric = likelihood_metric.at[1, i].set(GP_latent[i].likelihood_metric(key))
    jaccard_metric = jaccard_metric.at[1, i].set(GP_latent[i].jaccard_metric(ground_truth))

In [20]:
print(f'number_metric for Marginal {jnp.mean(number_metric[0])}')
print(f'number_metric for Latent {jnp.mean(number_metric[1])}')
print(f'location_metric for Marginal {jnp.mean(location_metric[0])}')
print(f'location_metric for Latent {jnp.mean(location_metric[1])}')
print(f'likelihood_metric for Marginal {jnp.mean(likelihood_metric[0])}')
print(f'likelihood_metric for Latent {jnp.mean(likelihood_metric[1])}')
print(f'jaccard_metric for Marginal {jnp.mean(jaccard_metric[0])}')
print(f'jaccard_metric for Latent {jnp.mean(jaccard_metric[1])}')
# print(location_metric)
# print(likelihood_metric)
# print(jaccard_metric)

number_metric for Marginal -0.11742856353521347
number_metric for Latent 0.1348571628332138
location_metric for Marginal 0.09127599000930786
location_metric for Latent 0.140904039144516
likelihood_metric for Marginal 242.09494018554688
likelihood_metric for Latent 290.868896484375
jaccard_metric for Marginal 0.954075813293457
jaccard_metric for Latent 0.9459378123283386


## Loading IHMM

In [4]:
name = 'Toy_dataset_RBF_3'
num_runs = 3

IHMM = []
for i in range(num_runs):
    IHMM.append(jnp.load(name + f'/IHMM/ihmm_trained_{i}.npy', allow_pickle = True)[()])

In [32]:
def metric1_IHMM(ihmm, true_number):

    def metric_func(max_states, z):
        estimated_states = z.flatten()
        uni_states = jnp.unique(estimated_states, size=max_states, fill_value=-1)
        found_states = jnp.sum(uni_states>-1)-1
        max_num = jnp.maximum(max_states - true_number, true_number)
        return (found_states - true_number)/max_num
    
    if isinstance(ihmm, list):
        result = []
        num_back_steps = 500
        for i, ihmm_ in enumerate(ihmm):
            max_states = ihmm_.max_states
            conv_z_arr = jnp.array(ihmm_.convergence['z'])
            estimated_states = conv_z_arr[-num_back_steps:].squeeze()
            result.append(jax.vmap(metric_func, in_axes=(None, 0))(max_states, estimated_states))
        result = jnp.array(result)
    else:
        result = metric_func(ihmm.max_states, ihmm.z)
    return jnp.mean(result)

def metric2_IHMM(ihmm, true_locations):

    if isinstance(ihmm, list):
        size = len(ihmm[0].z[0, :])
        x = np.linspace(0, 1, size)
        CP_loc = []
        num_back_steps = 500
        max_num = ihmm[0].max_states
        def find_CP(z):
            size = z.shape[0]
            size_arr = jnp.arange(size)
            carry, CP_loc = jax.lax.scan(lambda carry, y: (y[0], jnp.where(carry != y[0], (y[1] + 1)/(size+1), jnp.nan)), z[0], jnp.concatenate((z[:, None], size_arr[:, None]), axis = 1))
            return CP_loc
        
        for i, ihmm_ in enumerate(ihmm):
            conv_z_arr = jnp.array(ihmm_.convergence['z'])
            estimated_states = conv_z_arr[-num_back_steps:].squeeze()
            # print(estimated_states.shape)
            # for key, val in ihmm_.convergence.items():
            #     ihmm_.convergence[key] = jnp.stack(val, axis = 0)
            CP_loc.append(jax.vmap(find_CP, in_axes=(0, ))(estimated_states))
        CP_IHMM = jnp.reshape(jnp.array(CP_loc[-num_back_steps:]), (len(ihmm) * num_back_steps, -1))
        # print(CP_IHMM.shape)
        loc_metric = jnp.zeros(len(ihmm) * num_back_steps)
        for j, CP in enumerate(CP_IHMM):
            CP_new = CP[~jnp.isnan(CP)]
            found_states = len(CP_new)
            dist = np.zeros((len(true_locations), found_states))
            
            max_dist = 1
            for i, loc in enumerate(true_locations):
                true_locs = np.sort(x[np.argsort(np.abs(x - loc))[:2]])
                dist1 = (CP_new - true_locs[0])
                dist2 = (CP_new - true_locs[1])
                dist_comp = dist1 * dist2
                dist_min = np.minimum(np.abs(dist1), np.abs(dist2))
                dist_min[dist_comp < 0] = 0
                dist[i, :] = np.abs(dist_min)
            num_diff = np.abs(len(true_locations) - found_states)
            sorted_min_dist = np.sort(np.min(dist, axis = 1))
            if num_diff == 0:
                loc_metric = loc_metric.at[j].set(np.sum(sorted_min_dist)/max_num)
            else: 
                sorted_min_dist[-num_diff:] = max_dist
                loc_metric = loc_metric.at[j].set(np.sum(sorted_min_dist)/max_num)
        return jnp.mean(loc_metric)
    else:
        size = len(ihmm.z[0, :]) 
        x = np.linspace(0, 1, size)
        estimated_states = ihmm.z[0, :]
        CP_loc = np.where(estimated_states[:-1] != estimated_states[1:])[0] + 1
        CP_IHMM = x[CP_loc]
        found_states = len(CP_IHMM)
        dist = np.zeros((len(true_locations), found_states))
        max_num = ihmm.max_states
        max_dist = 1
        for i, loc in enumerate(true_locations):
            true_locs = np.sort(x[np.argsort(np.abs(x - loc))[:2]])
            dist1 = (CP_IHMM - true_locs[0])
            dist2 = (CP_IHMM - true_locs[1])
            dist_comp = dist1 * dist2
            dist_min = np.minimum(np.abs(dist1), np.abs(dist2))
            dist_min[dist_comp < 0] = 0
            dist[i, :] = np.abs(dist_min)
        num_diff = np.abs(len(true_locations) - found_states)
        sorted_min_dist = np.sort(np.min(dist, axis = 1))
        if num_diff == 0:
            return np.sum(sorted_min_dist)/max_num
        else: 
            sorted_min_dist[-num_diff:] = max_dist
            return np.sum(sorted_min_dist)/max_num

def metric3_IHMM(ihmm, y):

    def MVN_gen(ihmm_sub):
        mu = jax.vmap(lambda a, b: ihmm_sub['mu'].squeeze()[a] + ihmm_sub['A'].squeeze()[a] * b, in_axes=(1, 0))(ihmm_sub['z'], y[1:])
        cov = jax.vmap(lambda a: 1/ihmm_sub['invSigma'].squeeze()[a], in_axes=(0, ))(ihmm_sub['z'])
        MVN = dx.MultivariateNormalDiag(mu.squeeze(), cov.squeeze())
        log_prob = MVN.log_prob(y[1:])
        return log_prob

    if isinstance(ihmm, list):
        log_prob = []
        for i, ihmm_ in enumerate(ihmm):
            num_back_steps = 500
            for key, val in ihmm_.convergence.items():
                ihmm_.convergence[key] = jnp.stack(val, axis = 0)

            ihmm_param_in_axes = jax.tree_map(lambda l: 0, ihmm_.convergence)
            if ihmm_.convergence['nstates'].shape[0] != ihmm_.convergence['A'].shape[0]:
                ihmm_.convergence['nstates'] = jnp.insert(ihmm_.convergence['nstates'], 0, ihmm_.convergence['nstates'][0])
                ihmm_.convergence['z'] = jnp.append(ihmm_.convergence['z'][0][:, None], ihmm_.convergence['z'], axis = 0)
            log_prob.append(jax.vmap(MVN_gen, in_axes = (ihmm_param_in_axes, ))(ihmm_.convergence))
        log_prob = jnp.mean(jnp.array(log_prob))
    else:
        size = len(y)-1
        mu = np.zeros(size)
        cov = np.zeros(size)
        for i, z in enumerate(ihmm.z.squeeze()):
            # mu[i] = ihmm.theta['mu'].squeeze()[z]
            mu[i] = ihmm.theta['mu'].squeeze()[z] + ihmm.theta['A'].squeeze()[z] * y[i]
            cov[i] = 1/ihmm.theta['invSigma'].squeeze()[z]
        MVN = dx.MultivariateNormalDiag(mu, cov)
        log_prob = MVN.log_prob(y[1:])
    return log_prob


def metric4_IHMM(ihmm, true_locations):
    if isinstance(ihmm, list):
        size = len(ihmm[0].z[0, :])
    else:
        size = len(ihmm.z[0, :])
    true_matrix = np.zeros((size,size))
    for i in range(size):
            for j in range(size):
                if (np.sum(i < true_locations) == np.sum(j < true_locations)):
                    true_matrix[i, j] = 1
    
    if isinstance(ihmm, list):
        num_back_steps = 500
        J_index = []

        def gen_matrix(z):

            def check_zero(x):
                return jnp.where(x==0, jnp.nan, x)

            def check_side(x_, y_):
                return 1.0*(jnp.sum(jnp.less(x_, new_num)) == jnp.sum(jnp.less(y_, new_num)))

            size = z.shape[0]
            size_arr = jnp.arange(size)
            carry, CP_loc = jax.lax.scan(lambda carry, y: (y[0], jnp.where(carry != y[0], (y[1] + 1), jnp.nan)), z[0], jnp.concatenate((z[:, None], size_arr[:, None]), axis = 1))
            new_num = jax.vmap(lambda x: check_zero(x))(CP_loc)
            cov_matrix = jax.vmap(lambda x_: jax.vmap(lambda y_: check_side(x_, y_))(size_arr))(size_arr)
            return jnp.sum(1 - jnp.abs(cov_matrix - true_matrix))/(size**2)

        for i, ihmm_ in enumerate(ihmm):
            conv_z_arr = jnp.array(ihmm_.convergence['z'])
            estimated_states = conv_z_arr[-num_back_steps:].squeeze()
            J_index.append(jax.vmap(gen_matrix, in_axes=(0, ))(estimated_states))
        J_index = jnp.mean(jnp.array(J_index))
    
    else:
        estimated_states = ihmm.z[0, :]
        CP_loc = np.where(estimated_states[:-1] != estimated_states[1:])[0] + 2 
        cov_matrix = np.zeros((size,size))
        for i in range(size):
            for j in range(size):
                if (np.sum(i < CP_loc) == np.sum(j < CP_loc)):
                    cov_matrix[i, j] = 1
        J_index = np.sum(1 - np.abs(cov_matrix - true_matrix))/(size**2)

    return J_index

In [33]:
y = jnp.load(name + '/orig_data.npy')
ground_truth = jnp.load(name + f'/ground_truth.npy', allow_pickle = True)[()]
true_number = len(ground_truth['kernel']['num'])
true_locations = jnp.sort(ground_truth['kernel']['num']) * len(y)
# print(true_locations)

number_metric_IHMM = metric1_IHMM(IHMM, true_number)
print(number_metric_IHMM)
location_metric_IHMM = metric2_IHMM(IHMM, true_locations)
print(location_metric_IHMM)
likelihood_metric_IHMM = metric3_IHMM(IHMM, y)
print(likelihood_metric_IHMM)
jaccard_metric_IHMM = metric4_IHMM(IHMM, true_locations)
print(jaccard_metric_IHMM)

0.02838298
0.038140878
-122.69601
0.9557052


In [67]:
# key = jrnd.PRNGKey(1234)
number_metric_IHMM = jnp.zeros((3))
location_metric_IHMM = jnp.zeros((3))
likelihood_metric_IHMM = jnp.zeros((3))
jaccard_metric_IHMM = jnp.zeros((3))

y = jnp.load(name + '/orig_data.npy')
ground_truth = jnp.load(name + f'/ground_truth.npy', allow_pickle = True)[()]
true_number = len(ground_truth['kernel']['num'])
true_locations = jnp.sort(ground_truth['kernel']['num']) * len(y)
print(true_locations)

for i in range(num_runs):
    number_metric_IHMM = number_metric_IHMM.at[i].set(metric1_IHMM(IHMM[i], true_number))
    location_metric_IHMM = location_metric_IHMM.at[i].set(metric2_IHMM(IHMM[i], true_locations))
    likelihood_metric_IHMM = likelihood_metric_IHMM.at[i].set(metric3_IHMM(IHMM[i], y))
    jaccard_metric_IHMM = jaccard_metric_IHMM.at[i].set(metric4_IHMM(IHMM[i],true_locations))

[10. 30. 70.]


In [69]:
print(f'number_metric for IHMM {jnp.mean(number_metric_IHMM[0])}')
print(f'location_metric for IHMM {jnp.mean(location_metric_IHMM[0])}')
print(f'likelihood_metric for IHMM {jnp.mean(likelihood_metric_IHMM[0])}')
print(f'jaccard_metric for IHMM {jnp.mean(jaccard_metric_IHMM[0])}')

number_metric for IHMM 0.021276595070958138
location_metric for IHMM 0.03142857179045677
likelihood_metric for IHMM -145.8402099609375
jaccard_metric for IHMM 0.9618406295776367
